<a href="https://colab.research.google.com/github/s-c-soma/AdvanceDeeplearning-CMPE-297/blob/master/Practice/interactive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIF10

# Setup
First, install dependencies, import modules, set up paths, and download data.

In [3]:
!pip install -U tensorflow==2.0.0 pyarrow==0.14.1 tfx==0.15.0 grpcio==1.24.3 matplotlib==3.1.2
!pip freeze | grep -e tensorflow -e tfx -e pyarrow

     |████████████████████████████████| 86.3MB 52kB/s 
Requirement already up-to-date: pyarrow==0.14.1 in /usr/local/lib/python3.6/dist-packages (0.14.1)
     |████████████████████████████████| 512kB 49.7MB/s 
     |████████████████████████████████| 2.2MB 50.0MB/s 
     |████████████████████████████████| 13.1MB 48.8MB/s 
     |████████████████████████████████| 450kB 49.6MB/s 
     |████████████████████████████████| 3.8MB 50.6MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
     |████████████████████████████████| 225kB 56.1MB/s 
     |████████████████████████████████| 1.9MB 41.1MB/s 
     |████████████████████████████████| 1.4MB 35.5MB/s 
     |████████████████████████████████| 4.9MB 45.3MB/s 
     |████████████████████████████████| 2.3MB 50.8MB/s 
     |████████████████████████████████| 8.7MB 24.9MB/s 
     |████████████████████████████████| 471kB 54.3MB/s 
     |████████████████████████████████| 31.2MB 159kB/s 
     |████████████████████████████████| 10.4MB 46.9MB/s 
     |

pyarrow==0.14.1
tensorflow==2.0.0
tensorflow-addons==0.8.3
tensorflow-data-validation==0.15.0
tensorflow-datasets==4.0.1
tensorflow-estimator==2.0.1
tensorflow-gcs-config==2.3.0
tensorflow-hub==0.10.0
tensorflow-metadata==0.15.2
tensorflow-model-analysis==0.15.4
tensorflow-privacy==0.2.2
tensorflow-probability==0.11.0
tensorflow-serving-api==2.3.0
tensorflow-transform==0.15.0
tfx==0.15.0
tfx-bsl==0.15.3


# Import packages
We import necessary packages, including standard TFX component classes.

In [ ]:
import os
import absl
import tempfile
import urllib
import numpy as np
from matplotlib import pyplot as plt    
import matplotlib.patches as mpatches
import tensorflow as tf
import tensorflow_transform as tft
import tfx

from tfx.components import CsvExampleGen
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.utils.dsl_utils import external_input
from tfx.types import artifact_utils

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
import tempfile
from tfx.components.evaluator.component import Evaluator
from tfx.components.example_gen.import_example_gen.component import ImportExampleGen
from tfx.components.example_validator.component import ExampleValidator
from tfx.components.model_validator.component import ModelValidator
from tfx.components.pusher.component import Pusher
from tfx.components.schema_gen.component import SchemaGen
from tfx.components.statistics_gen.component import StatisticsGen
from tfx.components.trainer.component import Trainer
from tfx.components.transform.component import Transform
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import example_gen_pb2
from tfx.proto import evaluator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.utils.dsl_utils import external_input
from tfx.utils.types import get_split_uri

ModuleNotFoundError: ignored

In [ ]:
tf.enable_eager_execution()

In [ ]:
_root = '.'
_data_root = _root + '/data'
_serving_model_dir = os.path.join(tempfile.mkdtemp(),'serving_model/cifar-10')

In [ ]:
context = InteractiveContext()

# 1.ExampleGen

In [ ]:
input_split = example_gen_pb2.Input(splits=[
    example_gen_pb2.Input.Split(name='train', pattern='train.tfrecord'),
    example_gen_pb2.Input.Split(name='eval', pattern='test.tfrecord')
])
examples = external_input(_data_root)
example_gen = ImportExampleGen(input_base=examples, input_config=input_split)

context.run(example_gen)

# 2.StatisticsGen

In [ ]:
statistics_gen = StatisticsGen(input_data=example_gen.outputs['examples'])
 
context.run(statistics_gen)

In [ ]:
artifact_list = statistics_gen.outputs['output'].get()
train_artifact_uri = get_split_uri(artifact_list, 'train')
train_stats_path = os.path.join(train_artifact_uri, 'stats_tfrecord')
eval_artifact_uri = get_split_uri(artifact_list, 'eval')
eval_stats_path = os.path.join(eval_artifact_uri, 'stats_tfrecord')

In [ ]:
train_stats = tfdv.load_statistics(train_stats_path)
eval_stats = tfdv.load_statistics(eval_stats_path)

tfdv.visualize_statistics(lhs_statistics=eval_stats, rhs_statistics=train_stats,
                          lhs_name='EVAL_DATASET', rhs_name='TRAIN_DATASET')

# 3.SchemaGen

In [ ]:
infer_schema = SchemaGen(stats=statistics_gen.outputs['output'],
                         infer_feature_shape=True)

context.run(infer_schema)

In [ ]:
schema_dir = infer_schema.outputs['output'].get()[0].uri
schema_path = os.path.join(schema_dir, 'schema.pbtxt')

schema = tfdv.load_schema_text(schema_path)
tfdv.display_schema(schema)

# 4.ExampleValidator

In [ ]:
validate_stats = ExampleValidator(stats=statistics_gen.outputs['output'],
                                  schema=infer_schema.outputs['output'])

context.run(validate_stats)

In [ ]:
validation_dir = validate_stats.outputs['output'].get()[0].uri
anomalies_path = os.path.join(validation_dir, 'anomalies.pbtxt')

anomalies = tfdv.load_anomalies_text(anomalies_path)
tfdv.display_anomalies(anomalies)

# 5.Transform

In [ ]:
_module_file = os.path.join(_root, 'utils.py')

transform = Transform(
    input_data=example_gen.outputs['examples'],
    schema=infer_schema.outputs['output'],
    module_file=_module_file)

context.run(transform)

# 6.Trainer

In [ ]:
trainer = Trainer(
    module_file=_module_file,
    examples=transform.outputs['transformed_examples'],
    schema=infer_schema.outputs['output'],
    transform_output=transform.outputs['transform_output'],
    train_args=trainer_pb2.TrainArgs(num_steps=100),
    eval_args=trainer_pb2.EvalArgs(num_steps=50))
context.run(trainer)

# 7.Evaluator

In [ ]:
OVERALL_SLICE_SPEC = evaluator_pb2.SingleSlicingSpec()

ALL_SPECS = [
    OVERALL_SLICE_SPEC,
]

In [ ]:
model_analyzer = Evaluator(
    examples=example_gen.outputs['examples'],
    model_exports=trainer.outputs['output'],
    feature_slicing_spec=evaluator_pb2.FeatureSlicingSpec(
        specs=ALL_SPECS
    ))

context.run(model_analyzer)

In [ ]:
PATH_TO_RESULT = model_analyzer.outputs['output'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)
tfma.view.render_slicing_metrics(tfma_result)

In [ ]:
model_validator = ModelValidator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['output'])

context.run(model_validator)

# 8.Pusher

In [ ]:
pusher = Pusher(
    model_export=trainer.outputs['output'],
    model_blessing=model_validator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))

context.run(pusher)